In [1]:
#import necessary library
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import fdrcorrection
import operator

In [2]:
def GFS (series, theta1 = 0.05, theta2 = 0.15):
    rank = series.rank(method = 'average', ascending = False) # largest -> smallest
    high_rank = np.quantile(rank, theta1) # q1
    low_rank = np.quantile(rank, theta2) # q2
    
    output = []
    for i in rank:
        if i < high_rank:
            output.append(1)
        elif ((i >= high_rank) and (i <= low_rank)):
            temp = (i - low_rank) / (high_rank - low_rank)
            output.append(temp)
        else:
            output.append(0)
    return pd.Series(output)

In [3]:
def perform_GFS(dataframe, theta1 = 0.05, theta2 = 0.15):
    index = dataframe.index
    column = dataframe.columns
    
    df_GFS = pd.DataFrame()
    for columns in dataframe:
        sample = columns
        scored_list = GFS(dataframe[sample])
        df_GFS = pd.concat([df_GFS, scored_list], axis=1)
    
    df_GFS.index = index
    df_GFS.columns = column
    return (df_GFS)

In [4]:
#read data and metadata
GSE76275_withoutQN = pd.read_csv("GSE76275_withoutQN.csv", index_col=0)
metadata = pd.read_csv("GSE76275_withoutQN_metadata.csv", index_col= 0)

In [5]:
GSE76275_withoutQN

,GSM1974566,GSM1974567,GSM1974568,GSM1974569,GSM1974570,GSM1974571,GSM1974572,GSM1974573,GSM1974574,GSM1974575,...,GSM1978940,GSM1978941,GSM1978942,GSM1978943,GSM1978944,GSM1978945,GSM1978946,GSM1978947,GSM1978948,GSM1978949
ID,,,,,,,,,,,,,,,,,,,,,
1007_s_at,10.897690,11.630673,9.924386,10.244112,10.166529,10.103398,11.001749,10.079362,12.277516,9.615796,...,10.507550,10.391365,10.989400,10.610168,11.834494,11.289148,10.958152,11.798158,12.164959,10.682105
1053_at,8.889141,9.370075,7.987709,7.792648,8.037676,8.624163,8.320204,7.533532,8.314809,8.528061,...,6.618645,6.931382,7.743744,7.884446,7.157095,7.405463,7.747610,7.504686,8.662280,8.042492
117_at,7.672263,7.810757,7.529539,7.930183,6.984673,7.523831,6.304138,7.059384,7.796058,8.357557,...,6.738361,6.816242,7.393388,7.333748,6.874352,7.531275,7.223960,7.587937,7.438663,7.748113
121_at,9.354679,9.132152,8.965895,8.665752,8.238046,8.613339,7.910504,8.689870,8.801829,9.627912,...,8.606967,8.607288,9.043746,9.271634,8.645085,9.491085,9.476749,9.674215,9.068209,8.935698
1255_g_at,4.852996,4.925103,4.768807,4.517812,5.045035,4.476406,4.132390,4.439062,5.404663,4.945758,...,4.624186,4.586322,5.048083,4.893419,4.364246,4.221342,4.175757,5.382477,4.235272,4.129353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AFFX-ThrX-5_at,6.035713,5.664735,5.953573,5.729128,5.304806,5.076263,4.422419,4.864827,6.390706,6.500757,...,5.884376,6.125741,6.024650,5.508716,5.638332,4.311866,4.377143,5.942626,4.417946,4.402445
AFFX-ThrX-M_at,7.972801,7.234584,8.121668,7.876630,7.080817,6.182999,5.425722,6.058171,9.149894,9.044551,...,8.157140,8.488085,7.808982,7.491792,8.215128,4.164953,4.278036,7.080766,4.080618,4.359768
AFFX-TrpnX-3_at,4.308246,4.318494,4.465927,4.023585,4.097104,3.849739,3.355992,3.860198,4.184688,4.349899,...,4.007618,4.175178,4.622232,4.086933,3.768907,3.840029,3.622369,4.710942,3.733714,3.567453


In [6]:
TNBC_raw = GSE76275_withoutQN.T.loc[metadata['group'] == 'TN']
not_TNBC_raw = GSE76275_withoutQN.T.loc[metadata['group'] == 'not TN']

In [7]:
GSE76275_GFS = perform_GFS(GSE76275_withoutQN)

In [8]:
GSE76275_GFS = GSE76275_GFS.T

In [9]:
# Define two groups of breast cancer cells
TNBC = GSE76275_GFS.loc[metadata['group'] == 'TN']
not_TNBC = GSE76275_GFS.loc[metadata['group'] == 'not TN']

In [10]:
TNBC

ID,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
GSM1974566,0.966108,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,...,1.0,1.0,1.0,1.0,0.507389,0.0,0.0,0.0,0.0,0.0
GSM1974567,1.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,...,1.0,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0
GSM1974568,0.528606,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.015199,0.0,...,1.0,1.0,1.0,1.0,0.729250,0.0,0.0,0.0,0.0,0.0
GSM1974569,0.707301,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,0.574514,0.0,0.0,0.0,0.0,0.0
GSM1974570,0.962084,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.932820,0.0,...,1.0,1.0,1.0,1.0,0.639079,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1974759,1.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,0.115430,0.0,0.0,0.0,0.0,0.0
GSM1974760,1.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,0.074273,0.0,0.0,0.0,0.0,0.0
GSM1974761,1.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.973607,0.0,...,1.0,1.0,1.0,1.0,0.335827,0.0,0.0,0.0,0.0,0.0
GSM1974762,0.170483,0.0,0.0,0.000000,0.0,0.066595,0.0,0.0,0.183286,0.0,...,1.0,1.0,1.0,1.0,0.811739,0.0,0.0,0.0,0.0,0.0


In [11]:
not_TNBC

ID,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
GSM1978883,1.000000,0.0,0.0,0.000000,0.0,0.015565,0.0,0.0,0.101346,0.0,...,1.0,1.0,1.0,1.0,0.038611,0.0,0.0,0.0,0.0,0.0
GSM1978884,0.381552,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,...,1.0,1.0,1.0,1.0,0.274372,0.0,0.0,0.0,0.0,0.0
GSM1978885,1.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,0.427644,0.0,0.0,0.0,0.0,0.0
GSM1978886,0.319366,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,0.993726,0.0,0.0,0.0,0.0,0.0
GSM1978887,1.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,...,1.0,1.0,1.0,1.0,1.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1978945,1.000000,0.0,0.0,0.387588,0.0,0.015382,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0
GSM1978946,1.000000,0.0,0.0,0.395087,0.0,0.000000,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0
GSM1978947,1.000000,0.0,0.0,0.484344,0.0,0.000000,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,0.937941,0.0,0.0,0.0,0.0,0.0
GSM1978948,1.000000,0.0,0.0,0.000000,0.0,0.024710,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0


# Mann Whitney U-test & DEG Analysis (GSE76275)

In [15]:
#perform Mann Whiteney U-test (GSE76275)
import statsmodels.api as sm

GSE76275_result= pd.DataFrame({'Probe': [], 'p-value': [], 'q_value': [], 'DEG': []})
alpha = 0.05

U1, p = mannwhitneyu(TNBC, not_TNBC)
cols = TNBC.columns
rejected, q_value = fdrcorrection(p) # Benjaminin-HochBerg (FDR)

# Filter at p_adjusted lesser than 0.05
for i, col in enumerate(cols):
    #print(f'{col}: t = {t[i]:.5f}, with p-value = {p[i]:.5f}')
    if q_value[i] <= alpha:
    #    print ('We reject the null hypothesis H0. So, this gene is significantly differentially expressed.')
        a = 1
    else:
    #    print ('We do not reject the null hypothesis H0. So, this gene is  not significantly differentially expressed.')
        a = 0
    GSE76275_result= GSE76275_result.append({'Probe':col, 'q_value': q_value[i], 'p-value':p[i], 'DEG':a},ignore_index=True)

In [16]:
GSE76275_result

,Probe,p-value,q_value,DEG
0,1007_s_at,4.807991e-01,1.000000e+00,0.0
1,1053_at,1.675155e-02,1.336872e-01,0.0
2,117_at,1.042517e-01,5.419838e-01,0.0
3,121_at,9.005551e-01,1.000000e+00,0.0
4,1255_g_at,1.000000e+00,1.000000e+00,0.0
...,...,...,...,...
54670,AFFX-ThrX-5_at,1.000000e+00,1.000000e+00,0.0
54671,AFFX-ThrX-M_at,2.549650e-09,1.839078e-07,1.0
54672,AFFX-TrpnX-3_at,1.000000e+00,1.000000e+00,0.0
54673,AFFX-TrpnX-5_at,1.000000e+00,1.000000e+00,0.0


# Mann Whitney U-test DEGs (GSE76275)

In [121]:
# Rank and sort statistical test result with p-value
GSE76275_result_DEG = GSE76275_result[GSE76275_result['DEG'] == 1]

In [122]:
GSE76275_result_DEG = GSE76275_result_DEG.set_index('Probe')
GSE76275_result_DEG

,p-value,q_value,DEG
Probe,,,
1405_i_at,9.213959e-04,1.199460e-02,1.0
1552256_a_at,2.636931e-07,1.047017e-05,1.0
1552291_at,7.522568e-10,6.528514e-08,1.0
1552365_at,4.303810e-05,8.660965e-04,1.0
1552383_at,3.492096e-03,3.706666e-02,1.0
...,...,...,...
AFFX-r2-Bs-phe-M_at,1.125433e-10,1.286925e-08,1.0
AFFX-r2-Bs-thr-3_s_at,7.804247e-06,1.952848e-04,1.0
AFFX-r2-Bs-thr-M_s_at,6.705753e-08,3.166123e-06,1.0


In [123]:
log2_foldchange = (TNBC_raw.mean() -  not_TNBC_raw.mean()) / not_TNBC_raw.mean()
GSE76275_result_DEG['log2_FC'] = log2_foldchange

In [124]:
GSE76275_result_DEG

,p-value,q_value,DEG,log2_FC
Probe,,,,
1405_i_at,9.213959e-04,1.199460e-02,1.0,0.078713
1552256_a_at,2.636931e-07,1.047017e-05,1.0,0.072913
1552291_at,7.522568e-10,6.528514e-08,1.0,-0.081544
1552365_at,4.303810e-05,8.660965e-04,1.0,-0.124598
1552383_at,3.492096e-03,3.706666e-02,1.0,0.022923
...,...,...,...,...
AFFX-r2-Bs-phe-M_at,1.125433e-10,1.286925e-08,1.0,-0.138048
AFFX-r2-Bs-thr-3_s_at,7.804247e-06,1.952848e-04,1.0,-0.075158
AFFX-r2-Bs-thr-M_s_at,6.705753e-08,3.166123e-06,1.0,-0.088933


In [125]:
GSE76275_result_DEG = GSE76275_result_DEG.reset_index()
GSE76275_result_DEG

,Probe,p-value,q_value,DEG,log2_FC
0,1405_i_at,9.213959e-04,1.199460e-02,1.0,0.078713
1,1552256_a_at,2.636931e-07,1.047017e-05,1.0,0.072913
2,1552291_at,7.522568e-10,6.528514e-08,1.0,-0.081544
3,1552365_at,4.303810e-05,8.660965e-04,1.0,-0.124598
4,1552383_at,3.492096e-03,3.706666e-02,1.0,0.022923
...,...,...,...,...,...
5514,AFFX-r2-Bs-phe-M_at,1.125433e-10,1.286925e-08,1.0,-0.138048
5515,AFFX-r2-Bs-thr-3_s_at,7.804247e-06,1.952848e-04,1.0,-0.075158
5516,AFFX-r2-Bs-thr-M_s_at,6.705753e-08,3.166123e-06,1.0,-0.088933
5517,AFFX-ThrX-3_at,5.212198e-06,1.381371e-04,1.0,-0.080743


In [126]:
gene_name_doc = pd.read_csv("gene_name.csv")
gene_name_doc = gene_name_doc.drop_duplicates(subset=['initial_alias'], keep='first')
gene_name_doc = gene_name_doc.reset_index()

In [127]:
GSE76275_result_DEG['gene_name'] = gene_name_doc['name']
GSE76275_result_DEG['gene_desc'] = gene_name_doc['description']

In [128]:
GSE76275_result_DEG

,Probe,p-value,q_value,DEG,log2_FC,gene_name,gene_desc
0,1405_i_at,9.213959e-04,1.199460e-02,1.0,0.078713,CCL5,C-C motif chemokine ligand 5 [Source:HGNC Symb...
1,1552256_a_at,2.636931e-07,1.047017e-05,1.0,0.072913,SCARB1,scavenger receptor class B member 1 [Source:HG...
2,1552291_at,7.522568e-10,6.528514e-08,1.0,-0.081544,PIGX,phosphatidylinositol glycan anchor biosynthesi...
3,1552365_at,4.303810e-05,8.660965e-04,1.0,-0.124598,SCIN,scinderin [Source:HGNC Symbol;Acc:HGNC:21695]
4,1552383_at,3.492096e-03,3.706666e-02,1.0,0.022923,FAM71A,family with sequence similarity 71 member A [S...
...,...,...,...,...,...,...,...
5514,AFFX-r2-Bs-phe-M_at,1.125433e-10,1.286925e-08,1.0,-0.138048,None,None
5515,AFFX-r2-Bs-thr-3_s_at,7.804247e-06,1.952848e-04,1.0,-0.075158,None,None
5516,AFFX-r2-Bs-thr-M_s_at,6.705753e-08,3.166123e-06,1.0,-0.088933,None,None
5517,AFFX-ThrX-3_at,5.212198e-06,1.381371e-04,1.0,-0.080743,None,None


In [129]:
GSE76275_result_DEG_down = GSE76275_result_DEG.loc[GSE76275_result_DEG['log2_FC'] > 1]
GSE76275_result_DEG_up = GSE76275_result_DEG.loc[GSE76275_result_DEG['log2_FC'] > 1]

In [131]:
import os  
os.makedirs('C:/Users/yeyih/Desktop/GFS', exist_ok=True)  
GSE76275_result_DEG.to_csv('C:/Users/yeyih/Desktop/GFS/GSE76275(GFS)_DEG_latest.csv')